In [1]:
import os
import wandb
import torch
import json
import csv
import subprocess
import pandas as pd
import numpy as np
from tqdm import tqdm
from datasets import Dataset
from transformers import GPT2Tokenizer
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model

os.environ["CUDA_VISIBLE_DEVICES"] = "3"


# ---- I/O helpers ----
def load_data(train_path):
    with open(train_path, "r") as f:
        return [line.strip() for line in f]

def split_line(line):
    epi, tcr = line.split("$")
    return epi + "$", tcr

def save_to_csv_1(epis, tcrs, csv_file_path="tmp_epis_tcrs.csv"):
    with open(csv_file_path, mode="w", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=["Epitopes", "TCRs"])
        writer.writeheader()
        for epi, tcr in zip(epis, tcrs):
            writer.writerow({"Epitopes": epi, "TCRs": tcr})

def append_tmp_to_master(tmp_filename="tmp_epis_tcrs.csv", master_filename="all_results.csv"):
    master_exists = os.path.isfile(master_filename)
    tmp_df = pd.read_csv(tmp_filename)
    tmp_df.to_csv(master_filename, mode="a", header=not master_exists, index=False)

def collator(data):
    return {key: [d[key] for d in data] for key in data[0]}

def shannon_entropy(seq):
    import math
    from collections import Counter
    counts = Counter(seq)
    probs = [v / len(seq) for v in counts.values()]
    return -sum(p * math.log(p + 1e-9) for p in probs)

def longest_homopolymer(seq):
    if not seq:
        return 0
    longest, cur = 1, 1
    for a, b in zip(seq, seq[1:]):
        cur = cur + 1 if a == b else 1
        longest = max(longest, cur)
    return longest


# ---- Reward backend command builder ----
def build_reward_cmd(backend):
    if backend == "tcrbert":
        return (
            "conda run -n trl "
            "python rewards/tcrbert_bap_infer_neg.py "
            "--input-path tmp_epis_tcrs.csv "
            "--output-path tmp_epis_tcrs.csv "
            "--model-path rewards/tcrbert_bap_head_robust.pth"
        )
    elif backend == "catelmo":
        return (
            "conda run -n trl "
            "python rewards/catelmo_bap_infer_neg.py "
            "--input-path tmp_epis_tcrs.csv "
            "--output-path tmp_epis_tcrs.csv "
            "--model-path rewards/catelmo_bap_head_robust.pth"
        )
    else:  # cnn or lstm
        return (
            'bash -c "source activate tf26 && ' # conda activate /home/hmei7/workspace/.conda/envs/tf26 && '
            'python ../rewards/bap_ensemble/ensemble_bap.py '
            '--mode inference '
            '--testfile tmp_epis_tcrs.csv '
            f'--bap {backend}"'
        )

def run_reward_backend(backend):
    cmd = build_reward_cmd(backend)
    subprocess.run(cmd, shell=True, check=True)
    append_tmp_to_master()


# ---- Heuristic / reward post-processing ----
def apply_heuristic_penalties(reward_values, tcrs, cfg):
    """
    Apply heuristic penalties to reward_values in-place (returns a numpy array).
    reward_values: np.ndarray or list of floats
    tcrs: list of str
    cfg: HEURISTIC_CFG dict
    Returns: (np.ndarray reward_values_after, int bad_count)
    """
    rv = np.array(reward_values, dtype=float).copy()
    bad_count = 0
    for i, tcr in enumerate(tcrs):
        if not tcr or tcr == "WRONGFORMAT" or len(tcr) < 5:
            rv[i] = cfg["BAD_SEQ_PENALTY"]
            bad_count += 1
            continue
        L = len(tcr)
        H = shannon_entropy(tcr)
        R = longest_homopolymer(tcr)
        if not (cfg["L_MIN"] <= L <= cfg["L_MAX"]) or H < cfg["ENTROPY_FLOOR"] or R > cfg["RUN_MAX_ALLOWED"]:
            rv[i] = cfg["BAD_SEQ_PENALTY"]
            bad_count += 1
    # Clip large rewards
    rv = np.minimum(rv, cfg["MAX_REWARD"])
    return rv, bad_count


In [2]:
# ---- CONFIG / CONSTS ----
REWARD_BACKEND = "catelmo"  # options: "tcrbert", "catelmo", "cnn", "lstm"
BASE_CKPT_DIR = "ckpts/catelmo_infonce"
SAVE_EVERY = 5
os.makedirs(BASE_CKPT_DIR, exist_ok=True)
INCLUDE_HEURISTIC = False
# SPECIFITY = None # Placeholder for future use infonce or maxmargin


HEURISTIC_CFG = {
    "L_MIN": 9,
    "L_MAX": 21,
    "ENTROPY_FLOOR": 1.76,
    "RUN_MAX_ALLOWED": 3, # Max allowed homopolymer run (e.g., 'AAAA')
    "BAD_SEQ_PENALTY": -10.0,
    "MAX_REWARD": 10.0,
}

# ========================
# Configuration
# ========================
config = PPOConfig(
    model_name="/home/hmei7/workspace/tcr/models_gen/checkpoint-1600",
    learning_rate=1.41e-5, 
    batch_size=128,
    mini_batch_size=32, 
    ppo_epochs=2, # 2
    gradient_accumulation_steps=2,
    max_grad_norm=0.4,
    init_kl_coef=0.5, #[0.00, 0.02, 0.05, 1.0]
    adap_kl_ctrl=False,
    # target_kl=0.03,
    log_with="wandb",
    tracker_project_name="tcr-rlhf-goodhart",
)

# ========================
# Load Model and Tokenizer
# ========================
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
model_ref = create_reference_model(model)
tokenizer = GPT2Tokenizer.from_pretrained("/home/hmei7/workspace/tcr/models_gen/aa_tokenizer_trained")
tokenizer.pad_token = tokenizer.eos_token


dataset_lines = load_data('data/epi_training.txt')
epis, tcrs = [], []
for line in dataset_lines:
    epi, tcr = split_line(line)
    epis.append(epi)
    tcrs.append(tcr)
epis = list(set(epis))  # deduplicate epitopes
print(f"Loaded {len(epis)} epitopes:")
my_dataset = {"epis": epis}
dataset = Dataset.from_dict(my_dataset)
dataset.set_format("pytorch")


# Tokenization & Query Construction
dataset = dataset.map(
    lambda x: {"input_ids": tokenizer.encode(x["epis"], return_tensors="pt")[0, :64].to(model.pretrained_model.device)},
    batched=False,
)
dataset = dataset.map(
    lambda x: {"query": tokenizer.decode(x["input_ids"])},
    batched=False
)

# ========================
# Initialize PPO Trainer
# ========================
ppo_trainer = PPOTrainer(config, model, model_ref, tokenizer, dataset=dataset, data_collator=collator)


# ========================
# Generation Configuration
# ========================
generation_kwargs = {
    "min_length": 2,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "eos_token_id": tokenizer.eos_token_id,
    # "max_length": 72,
    "max_new_tokens": 42,  
}

# ========================
# Training Loop
# ========================
global_step = 0
for epoch in tqdm(range(10)):
    for batch in ppo_trainer.dataloader:
        
        # ---- batch generations ----
        query_tensors = batch["input_ids"]                 # List[Tensor], variable lengths
        queries_text   = batch.get("query", None)          # optional, if you want raw text

        with torch.no_grad():                              
            gen_full = ppo_trainer.generate(list(query_tensors), **generation_kwargs)
        prompt_lens = [int(q.shape[-1]) for q in query_tensors]
        response_tensors = [g[p_len:] for g, p_len in zip(gen_full, prompt_lens)]
        batch["response"] = tokenizer.batch_decode(response_tensors, skip_special_tokens=False)
        

        # ---- extract epi/tcr ----
        epis = [tokenizer.decode(r.squeeze())[:-2] for r in query_tensors]
        tcrs = [resp.split('<EOS>')[0] for resp in batch["response"]]
        special_tokens = ["<PAD>", "<tcr>", "<eotcr>", "[CLS]", "[BOS]", "[MASK]", "[SEP]", "<epi>", "<eoepi>", "$", "<unk>"]
        tcrs = ['WRONGFORMAT' if (not s or any(tok in s for tok in special_tokens)) else s for s in tcrs]
        save_to_csv_1(epis, tcrs, 'tmp_epis_tcrs.csv')

        
        # ---- run ALL reward backends sequentially ----
        run_reward_backend(REWARD_BACKEND)

        
        # ---- Reward config (set once, e.g., near other configs) ----
        REWARD_MODE = os.environ.get("REWARD_MODE", "infonce")   # "max_margin" | "infonce"
        TAU = float(os.environ.get("REWARD_TAU", "0.25"))           # temperature for infonce (logit units)
        Z_SCORE_INFO = True  
        # ---- I/O reward values (max-margin or InfoNCE) ----
        df_rewards = pd.read_csv("tmp_epis_tcrs.csv")

        pos = df_rewards["logits"].astype(float).to_numpy()

        if "logits_neg" in df_rewards.columns:
            neg_lists = df_rewards["logits_neg"].fillna("[]").map(json.loads)
        else:
            neg_lists = [None] * len(pos)  # keep same shape; no negatives

        if REWARD_MODE == "max_margin":
            # r = pos_logit - max(neg_logits); fallback to pos if no negatives present
            reward_values = []
            for p, ns in zip(pos, neg_lists):
                if ns and len(ns) > 0:
                    reward_values.append(p - max(ns))
                else:
                    reward_values.append(p)

        elif REWARD_MODE == "infonce":
            # r = log_softmax(pos / tau) among [pos] + negs (≤ 0); z-score for PPO stability
            info_vals = []
            raw_margins = []  # optional diagnostic (comparable to old runs)
            for p, ns in zip(pos, neg_lists):
                if ns and len(ns) > 0:
                    x = np.array([p] + ns, dtype=np.float64) / max(TAU, 1e-6)
                    x = x - x.max()                   # stabilize
                    lse = np.log(np.exp(x).sum())     # logsumexp
                    info_vals.append(float(x[0] - lse))
                    raw_margins.append(p - max(ns))
                else:
                    # no negatives -> neutral InfoNCE; keep margin comparable to pos
                    info_vals.append(0.0)
                    raw_margins.append(p)

            r = np.array(info_vals, dtype=np.float32)
            if Z_SCORE_INFO:
                mu, sd = r.mean(), r.std()
                r = (r - mu) / (sd + 1e-8)

            reward_values = r.tolist()


            
        rewards_bap = [torch.tensor(float(v), dtype=torch.float32) for v in reward_values]

        # PPO Step
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards_bap)
        ppo_trainer.log_stats(stats, batch, rewards_bap)
        
        # ---- save checkpoints ----
        global_step += 1
        if global_step % SAVE_EVERY == 0 and ppo_trainer.accelerator.is_main_process:
            ckpt_dir = os.path.join(BASE_CKPT_DIR, f"step_{global_step:04d}")
            os.makedirs(ckpt_dir, exist_ok=True)
            ppo_trainer.save_pretrained(ckpt_dir)
            tokenizer.save_pretrained(ckpt_dir)
            print(f"[Checkpoint] Saved to {ckpt_dir}")

    print(f'\nEpoch {epoch}:')
    print(tcrs[:20])


/home/hmei7/workspace/.conda/envs/bap_attack/lib/python3.10/site-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(
/home/hmei7/workspace/.conda/envs/bap_attack/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/hmei7/workspace/.conda/envs/bap_attack/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For mo

Loaded 786 epitopes:


Map:   0%|          | 0/786 [00:00<?, ? examples/s]

Map:   0%|          | 0/786 [00:00<?, ? examples/s]

/home/hmei7/workspace/.conda/envs/bap_attack/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pengfeiz. Use `wandb login --relogin` to force relogin


...

43
Processing neg_1...
Processing neg_2...
Processing neg_3...
Processing neg_4...



/mnt/disk11/user/pzhang84/anaconda3/envs/trl/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/mnt/disk11/user/pzhang84/anaconda3/envs/trl/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

Map (num_proc=4): 100%|██████████| 128/128 [00:00<00:00, 454.86 examples/s]

100%|██████████| 1/1 [00:00<00:00,  3.14it/s]

Map (num_proc=4): 100%|██████████| 128/128 [00:00<00:00, 593.99 examples/s]

100%|██████████| 1/1 [00:00<00:00,  6.91it/s]

Map (num_proc=4): 100%|██████████| 128/128 [00:00<00:00, 570.56 examples/s] 

100%|██████████| 1/1 [00:00<00:00,  7.03it/s]

Map (num_proc=4): 100%|█

43
Processing neg_1...
Processing neg_2...
Processing neg_3...
Processing neg_4...



/mnt/disk11/user/pzhang84/anaconda3/envs/trl/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/mnt/disk11/user/pzhang84/anaconda3/envs/trl/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

Map (num_proc=4): 100%|██████████| 128/128 [00:00<00:00, 480.70 examples/s]

100%|██████████| 1/1 [00:00<00:00,  2.94it/s]

Map (num_proc=4): 100%|██████████| 128/128 [00:00<00:00, 589.32 examples/s]

100%|██████████| 1/1 [00:00<00:00,  6.32it/s]

Map (num_proc=4): 100%|██████████| 128/128 [00:00<00:00, 518.02 examples/s] 

100%|██████████| 1/1 [00:00<00:00,  6.12it/s]

Map (num_proc=4): 100%|█

[Checkpoint] Saved to ckpts/catelmo_infonce/step_0060

Epoch 9:
['CASSDGLGDTQYF', 'CASSQSPAEAFF', 'CASQYPSEAFF', 'CASSLGNTEAFF', 'CASSLQRADTQYF', 'CASNGTISDTQYF', 'CASSLSFETQYF', 'CASSIGADTQYF', 'CASSTSGSYNEQFF', 'CASSDPYEQYF', 'CASSLAGTEAFF', 'CASSQQLQYF', 'CASSQGPNTEAFF', 'CASSQGPSAETQYF', 'CASSIQLAETQYF', 'CASSQTTGELF', 'CASSMGETQYF', 'CASSLSDTQYF', 'CASSLEPHEEETQYF', 'CASSEPANTEAFF']
